In [267]:
import base64
import requests
import json
import pandas as pd
import flat_table
from config import consumer_key,consumer_secret

In [268]:
consumer_key_secret = consumer_key+":"+consumer_secret
consumer_key_secret_enc = base64.b64encode(consumer_key_secret.encode()).decode()

In [269]:
headersAuth = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': 'Basic '+ str(consumer_key_secret_enc)
}

data = {
    'grant_type': 'client_credentials',
    'scope' : 'product.compact'
}

## Authentication request
response = requests.post('https://api.kroger.com/v1/connect/oauth2/token', headers=headersAuth, data=data, verify=True)
j = response.json()

access_token = (j['access_token'])


In [270]:
def get_milk_price(): 
    
    # initialization
    page = 1
    start=1
    output = []
    
    
    while page<21: 

        # run the request
        headersAPI = {
            'accept': 'application/json',
            'Authorization': 'Bearer '+access_token
            }

        lc=''
        y = 1
        
        if y == 1:
            lc = '03400312'
        elif y == 2:
            lc = '03400334'
        elif y == 3:
            lc = '03400121'
        
        
        # run the request
              
        http = "https://api.kroger.com/v1/products?filter.term={{"
        term = '"milk"'
        http2 = "}}&filter.limit="
        limit=50
        http3 = "&filter.start="
        start = 1
        http4 = "&filter.locationId="
        loc = lc
      

        url = http+term+http2+str(limit)+http3+str(start)+http4+loc

        response = requests.get(url,headers=headersAPI, verify=True).json()       
                
        
        # add response data to output
        output += response['data']
        
        # handle pagination
        page += 1
        start+=50
    return output    



In [271]:
milk_price_list = get_milk_price()

with open('milk_price_list.json', 'w') as f:
    json.dump(milk_price_list, f)

In [272]:
milk_price_list

[{'productId': '0001111040101',
  'upc': '0001111040101',
  'aisleLocations': [{'bayNumber': '41',
    'description': 'Back Wall',
    'number': '152',
    'numberOfFacings': '2',
    'sequenceNumber': '12',
    'side': 'R',
    'shelfNumber': '4',
    'shelfPositionInBay': '1'},
   {'bayNumber': '42',
    'description': 'Back Wall',
    'number': '152',
    'numberOfFacings': '4',
    'sequenceNumber': '12',
    'side': 'R',
    'shelfNumber': '3',
    'shelfPositionInBay': '1'},
   {'bayNumber': '42',
    'description': 'Back Wall',
    'number': '152',
    'numberOfFacings': '4',
    'sequenceNumber': '12',
    'side': 'R',
    'shelfNumber': '4',
    'shelfPositionInBay': '1'},
   {'bayNumber': '41',
    'description': 'Back Wall',
    'number': '152',
    'numberOfFacings': '4',
    'sequenceNumber': '12',
    'side': 'R',
    'shelfNumber': '3',
    'shelfPositionInBay': '1'}],
  'brand': 'Kroger',
  'categories': ['Dairy'],
  'description': 'Kroger® Vitamin D Whole Milk',
  'ima

In [315]:
#Normalize the json data into a data frame and expand the items category to pick up the container size
df = pd.json_normalize(milk_price_list,"items",["productId", "categories","description","upc","brand"])

df.head(3)

,itemId,favorite,size,soldBy,fulfillment.curbside,fulfillment.delivery,fulfillment.inStore,fulfillment.shipToHome,price.regular,price.promo,productId,categories,description,upc,brand
0,0001111040101,False,1 gal,Unit,True,True,True,False,2.99,0.0,0001111040101,[Dairy],Kroger® Vitamin D Whole Milk,0001111040101,Kroger
1,0001111041600,False,1/2 gal,Unit,True,True,True,False,1.99,0.0,0001111041600,[Dairy],Kroger® 2% Reduced Fat Milk,0001111041600,Kroger
2,0001111041660,False,1 gal,Unit,True,True,True,False,2.99,0.0,0001111041660,[Dairy],Kroger® 1% Lowfat Milk,0001111041660,Kroger


In [316]:
#Select only the columns we're wanting to keep
df = df[['productId','categories','brand','description','size','price.regular','price.promo']]

df

,productId,categories,brand,description,size,price.regular,price.promo
0,0001111040101,[Dairy],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.00
1,0001111041600,[Dairy],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.00
2,0001111041660,[Dairy],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.00
3,0001111040601,[Dairy],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.00
4,0001111042315,[Dairy],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.00
...,...,...,...,...,...,...,...
995,0085508800546,"[Dairy, Natural & Organic]",Maple Hill,Maple Hill Organic 100% Grassfed Whole Milk,1/2 gal,7.29,5.79
996,0001111004990,"[Dairy, Natural & Organic]",Simple Truth Organic,Simple Truth Organic® 100% Grass-Fed 2% Milk,64 fl oz,5.29,0.00
997,0076352800022,[Dairy],Promised Land,Promised Land Dairy Very Berry Strawberry Whol...,28 fl oz,3.19,0.00
998,0081547301032,[Dairy],Borden,Borden Whole Vitamin D Milk,1 gal,3.99,0.00


In [317]:
#Add date and Store Id
df["Date"] = '02/08/2021'
df["Store_Id"]= '03400312'

df.head(3)

,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id
0,0001111040101,[Dairy],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,03400312
1,0001111041600,[Dairy],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,03400312
2,0001111041660,[Dairy],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,03400312


In [336]:
#Add image
#df['image'] = df.agg('https://www.kroger.com/product/images/xlarge/front/{[productId]}'.format, axis=1)

df['image'] = 'https://www.kroger.com/product/images/xlarge/front/'+df["productId"].astype(str)



df.head(3)

,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0001111040101,[Dairy],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,03400312,https://www.kroger.com/product/images/xlarge/front/0001111040101
1,0001111041600,[Dairy],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,03400312,https://www.kroger.com/product/images/xlarge/front/0001111041600
2,0001111041660,[Dairy],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,03400312,https://www.kroger.com/product/images/xlarge/front/0001111041660
